In [ ]:

from gurobipy import Model, GRB, quicksum
from data_generation import generate_data
from visualizations import plot_tour_graph
import itertools
import numpy as np

def calculate_M(travel_time):
    num_cities = len(travel_time)
    M = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                M[i, j] = max(
                    sum(travel_time[u, v] for u, v in zip(path[:-1], path[1:]))
                    for path in itertools.permutations(range(num_cities), num_cities)
                    if path[0] == i and path[-1] == j
                )
    return M

def solve_tsptw_with_M(num_cities, seed=42):
    coords, time_windows, travel_time = generate_data(num_cities, seed)
    M = calculate_M(travel_time)
    model = Model()

    # Decision variables
    x = model.addVars(num_cities, num_cities, vtype=GRB.BINARY, name="x")
    z = model.addVars(num_cities, vtype=GRB.CONTINUOUS, name="z")
    w = model.addVars(num_cities, vtype=GRB.CONTINUOUS, name="w")
    h = model.addVars(num_cities, vtype=GRB.CONTINUOUS, name="h")

    # Objective function
    model.setObjective(
        quicksum(travel_time[i, j] * x[i, j] for i in range(num_cities) for j in range(num_cities))
        + quicksum(w[i] + h[i] for i in range(num_cities)),
        GRB.MINIMIZE,
    )

    # Constraints
    for i in range(num_cities):
        model.addConstr(quicksum(x[i, j] for j in range(num_cities)) == 1, name=f"Outflow_{i}")
        model.addConstr(quicksum(x[j, i] for j in range(num_cities)) == 1, name=f"Inflow_{i}")
        model.addConstr(x[i, i] == 0, name=f"NoSelfLoop_{i}")
        model.addConstr(w[i] >= time_windows[i, 0] - z[i], name=f"WaitingTime_{i}")
        model.addConstr(h[i] >= z[i] - time_windows[i, 1], name=f"LatePenalty_{i}")

    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                model.addConstr(z[j] >= z[i] + travel_time[i, j] * x[i, j] - M[i, j] * (1 - x[i, j]), name=f"Subtour_{i}_{j}")

    model.optimize()

    if model.status == GRB.INFEASIBLE:
        print("Model is infeasible. Attempting feasibility relaxation...")
        model.feasRelaxS(0, False, False, True)
        model.optimize()

    if model.status == GRB.OPTIMAL:
        route = np.zeros((num_cities, num_cities))
        for i in range(num_cities):
            for j in range(num_cities):
                if x[i, j].X > 0.5:
                    route[i, j] = 1
        return coords, route, model.objVal
    else:
        return coords, None, None

# Example usage
coords, route, obj_val = solve_tsptw_with_M(10)
if route is not None:
    plot_tour_graph(coords, route, title="TSPTW Solution with M for 10 Cities")
